<a href="https://www.kaggle.com/code/eslamfouad/apd-prediction-using-timeseries?scriptVersionId=120809153" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amp-parkinsons-disease-progression-prediction/train_proteins.csv
/kaggle/input/amp-parkinsons-disease-progression-prediction/train_clinical_data.csv
/kaggle/input/amp-parkinsons-disease-progression-prediction/public_timeseries_testing_util.py
/kaggle/input/amp-parkinsons-disease-progression-prediction/supplemental_clinical_data.csv
/kaggle/input/amp-parkinsons-disease-progression-prediction/train_peptides.csv
/kaggle/input/amp-parkinsons-disease-progression-prediction/amp_pd_peptide/competition.cpython-37m-x86_64-linux-gnu.so
/kaggle/input/amp-parkinsons-disease-progression-prediction/amp_pd_peptide/__init__.py
/kaggle/input/amp-parkinsons-disease-progression-prediction/example_test_files/sample_submission.csv
/kaggle/input/amp-parkinsons-disease-progression-prediction/example_test_files/test_proteins.csv
/kaggle/input/amp-parkinsons-disease-progression-prediction/example_test_files/test_peptides.csv
/kaggle/input/amp-parkinsons-disease-progression-prediction/example_test

In [2]:
train_peptides = pd.read_csv('/kaggle/input/amp-parkinsons-disease-progression-prediction/train_peptides.csv')
train_proteins = pd.read_csv('/kaggle/input/amp-parkinsons-disease-progression-prediction/train_proteins.csv')
train_clinical = pd.read_csv('/kaggle/input/amp-parkinsons-disease-progression-prediction/train_clinical_data.csv')
test_peptides = pd.read_csv('/kaggle/input/amp-parkinsons-disease-progression-prediction/example_test_files/test_peptides.csv') # test_xxx.csv 为题给的样例数据
test_proteins = pd.read_csv('/kaggle/input/amp-parkinsons-disease-progression-prediction/example_test_files/test_proteins.csv')
test_clinical = pd.read_csv('/kaggle/input/amp-parkinsons-disease-progression-prediction/example_test_files/test.csv')

In [3]:
print("Train_peptides.csv shape: ", train_peptides.shape)
train_peptides.sample(5)

Train_peptides.csv shape:  (981834, 6)


,visit_id,visit_month,patient_id,UniProt,Peptide,PeptideAbundance
468061,10138_24,24,10138,P13611,AQC(UniMod_4)GGGLLGVR,5079.4
354879,27715_12,12,27715,P02751,WLPSSSPVTGYR,359264.0
114456,44001_0,0,44001,P04156,VVEQMC(UniMod_4)ITQYER,384267.0
430299,59550_12,12,59550,O00533,TLKIENVSYQDKGNYR,23707.9
614915,7886_36,36,7886,O00533,KPQSAVYSTGSNGILLC(UniMod_4)EAEGEPQPTIK,22986.6


In [4]:
patient_id_list = list(set(train_peptides["patient_id"].values))
print(">>> Num of Patients: ", len(patient_id_list))
print(">>> Patients ID List(Top12): ", patient_id_list[:12])
visit_times = []

for patient_id in patient_id_list:
    visited_month_list = list(set(train_peptides[train_peptides["patient_id"] == patient_id]["visit_month"]))
    visit_times.append(len(visited_month_list))

print(">>> Visit_time_count: ")
print(pd.Series(visit_times).value_counts())

>>> Num of Patients:  248
>>> Patients ID List(Top12):  [17414, 40967, 49672, 55302, 14344, 5645, 38419, 42003, 49683, 61974, 65043, 40980]
>>> Visit_time_count: 
3    74
4    65
5    46
6    33
7    12
8     7
9     6
2     4
1     1
dtype: int64


In [5]:
example_patient_id = 17414
df_example = train_peptides[train_peptides["patient_id"] == example_patient_id]
visit_m = sorted(list(set(df_example["visit_month"].values))) # [0, 36, 12, 48, 24, 60]
print(">>> Patient_{} Analysis: ".format(example_patient_id))

for m in visit_m:
    m_df = df_example[df_example["visit_month"] == m]
    prot_list = list(set(m_df["UniProt"].values))
    peptide_list = list(set(m_df["Peptide"].values))
    print(" > visit_month: ", m)
    print("     num of protein: ", len(prot_list))
    print("     num of peptide: ", len(peptide_list))

>>> Patient_17414 Analysis: 
 > visit_month:  0
     num of protein:  209
     num of peptide:  859
 > visit_month:  12
     num of protein:  200
     num of peptide:  809
 > visit_month:  24
     num of protein:  207
     num of peptide:  862
 > visit_month:  36
     num of protein:  213
     num of peptide:  903
 > visit_month:  48
     num of protein:  211
     num of peptide:  894
 > visit_month:  60
     num of protein:  196
     num of peptide:  760


In [6]:
print(train_proteins.shape)
train_proteins.sample(5)

(232741, 5)


,visit_id,visit_month,patient_id,UniProt,NPX
123127,27468_24,24,27468,P02647,8806620.0
164758,47171_36,36,47171,P04217,1368910.0
212745,28327_60,60,28327,P43121,50550.8
213377,34182_60,60,34182,P31997,295518.0
149282,16566_36,36,16566,Q14515,371453.0


In [7]:
patient_id_list = list(set(train_proteins["patient_id"].values))
print(">>> Num of Patients: ", len(patient_id_list))
print(">>> Patients ID List(Top12): ", patient_id_list[:12])
visit_times = []

for patient_id in patient_id_list:
    visited_month_list = list(set(train_proteins[train_proteins["patient_id"] == patient_id]["visit_month"]))
    visit_times.append(len(visited_month_list))
    
print(">>> visit_time_count: ")
print(pd.Series(visit_times).value_counts())

>>> Num of Patients:  248
>>> Patients ID List(Top12):  [17414, 40967, 49672, 55302, 14344, 5645, 38419, 42003, 49683, 61974, 65043, 40980]
>>> visit_time_count: 
3    74
4    65
5    46
6    33
7    12
8     7
9     6
2     4
1     1
dtype: int64


In [8]:
example_patient_id = 17414
df_example = train_proteins[train_proteins["patient_id"] == example_patient_id]
visit_m = sorted(list(set(df_example["visit_month"].values)))
print(">>> Patient_ {} Analysis: ".format(example_patient_id))

for m in visit_m:
    m_df = df_example[df_example["visit_month"] == m]
    prot_list = list(set(m_df["UniProt"].values))
    print(" > visit_month: ", m)
    print("       num of protein: ", len(prot_list))

>>> Patient_ 17414 Analysis: 
 > visit_month:  0
       num of protein:  209
 > visit_month:  12
       num of protein:  200
 > visit_month:  24
       num of protein:  207
 > visit_month:  36
       num of protein:  213
 > visit_month:  48
       num of protein:  211
 > visit_month:  60
       num of protein:  196


In [9]:
print(train_clinical.shape)
train_clinical.sample(5)

(2615, 8)


,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication
879,20707_6,20707,6,1.0,0.0,1.0,NaN,NaN
2016,51708_108,51708,108,8.0,7.0,44.0,0.0,Off
1685,41628_60,41628,60,2.0,0.0,4.0,NaN,NaN
888,20707_54,20707,54,3.0,0.0,12.0,NaN,NaN
303,10138_6,10138,6,2.0,5.0,15.0,0.0,On


In [10]:
patient_id_list = list(set(train_clinical["patient_id"].values))
print(">>> Num of Patients: ", len(patient_id_list))
print(">>> Patients ID List(Top12): ", patient_id_list[:12])

>>> Num of Patients:  248
>>> Patients ID List(Top12):  [17414, 40967, 14344, 49672, 55302, 5645, 38419, 40980, 42003, 49683, 61974, 65043]


In [11]:
test_peptides.head()
test_proteins.head()

,visit_id,visit_month,patient_id,UniProt,NPX,group_key
0,50423_0,0,50423,O00391,33127.90,0
1,50423_0,0,50423,O00533,490742.00,0
2,50423_0,0,50423,O00584,43615.30,0
3,50423_0,0,50423,O14773,16486.60,0
4,50423_0,0,50423,O14791,2882.42,0


In [12]:
test_clinical.head()

,visit_id,visit_month,patient_id,updrs_test,row_id,group_key
0,3342_0,0,3342,updrs_1,3342_0_updrs_1,0
1,3342_0,0,3342,updrs_2,3342_0_updrs_2,0
2,3342_0,0,3342,updrs_3,3342_0_updrs_3,0
3,3342_0,0,3342,updrs_4,3342_0_updrs_4,0
4,50423_0,0,50423,updrs_1,50423_0_updrs_1,0


In [13]:
sample_submission = pd.read_csv('/kaggle/input/amp-parkinsons-disease-progression-prediction/example_test_files/sample_submission.csv')
sample_submission.head()

,prediction_id,rating,group_key
0,3342_0_updrs_1_plus_0_months,0,0
1,3342_0_updrs_1_plus_6_months,0,0
2,3342_0_updrs_1_plus_12_months,0,0
3,3342_0_updrs_1_plus_24_months,0,0
4,3342_0_updrs_2_plus_0_months,0,0


In [14]:
supp = pd.read_csv('/kaggle/input/amp-parkinsons-disease-progression-prediction/supplemental_clinical_data.csv')
supp.head()

,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication
0,35_0,35,0,5.0,3.0,16.0,0.0,NaN
1,35_36,35,36,6.0,4.0,20.0,0.0,NaN
2,75_0,75,0,4.0,6.0,26.0,0.0,NaN
3,75_36,75,36,1.0,8.0,38.0,0.0,On
4,155_0,155,0,NaN,NaN,0.0,NaN,NaN


In [15]:
# Metric: SMAPE
def smape(y_true, y_pred):
    smap = np.zeros(len(y_true))
    
    num = np.abs(y_true - y_pred)
    dem = ((np.abs(y_true) + np.abs(y_pred)) / 2)
    
    pos_ind = dem != 0
    smap[pos_ind] = num[pos_ind] / dem[pos_ind]
    
    return 100 * np.mean(smap)

In [16]:
train_clinical.isna().sum()

visit_id                                  0
patient_id                                0
visit_month                               0
updrs_1                                   1
updrs_2                                   2
updrs_3                                  25
updrs_4                                1038
upd23b_clinical_state_on_medication    1327
dtype: int64

In [17]:
estimates = {}
months = train_clinical.visit_month.unique()
targets = ['updrs_1', 'updrs_2', 'updrs_3', 'updrs_4']
for m in months:
    for target in targets:
        t = train_clinical[train_clinical.visit_month==m][f'{target}'].dropna().values
        if len(t) >= 200:
            s = []
            best_threshold = 0
            best_smape = 200
            for i in np.arange(0, 30, 0.1):
                score = smape(t, np.array([i for _ in range(len(t))]))
                s.append(score)
                if score < best_smape:
                    best_smape = score
                    best_threshold = i
        else:
            best_threshold = np.median(t)
        estimates[(m, target)] = best_threshold

for i in range(sorted(months)[-1]+1):
    for target in targets:
        if (i, target) not in estimates:
            estimates[(i, target)] = estimates[(i-1, target)]
            
validation_x = []
validation_y = []

for id, row in train_clinical.iterrows():
    for t in targets:
        if row[f'{t}']>=0:
            validation_x.append((row.visit_month, t))
            validation_y.append(row[f'{t}'])
            
smape(validation_y, pd.Series(validation_x).map(estimates).values)

76.36180750178882

In [18]:
train_clinical.isna().sum()

visit_id                                  0
patient_id                                0
visit_month                               0
updrs_1                                   1
updrs_2                                   2
updrs_3                                  25
updrs_4                                1038
upd23b_clinical_state_on_medication    1327
dtype: int64

In [19]:
# Baseline 提出的简单方法
estimates = {}
months = train_clinical.visit_month.unique()
targets = ['updrs_1', 'updrs_2', 'updrs_3', 'updrs_4']
for m in months:
    for target in targets:
        t = train_clinical[train_clinical.visit_month==m][f'{target}'].dropna().values
        if len(t) >= 200:
            s = []
            best_threshold = 0
            best_smape = 200
            for i in np.arange(0, 30, 0.1):
                score = smape(t, np.array([i for _ in range(len(t))]))
                s.append(score)
                if score < best_smape:
                    best_smape = score
                    best_threshold = i
        else:
            best_threshold = np.median(t)
        estimates[(m, target)] = best_threshold

for i in range(sorted(months)[-1]+1):
    for target in targets:
        if (i, target) not in estimates:
            estimates[(i, target)] = estimates[(i-1, target)]
            
validation_x = []
validation_y = []

for id, row in train_clinical.iterrows():
    for t in targets:
        if row[f'{t}']>=0:
            validation_x.append((row.visit_month, t))
            validation_y.append(row[f'{t}'])
            
smape(validation_y, pd.Series(validation_x).map(estimates).values)

76.36180750178882

In [20]:
import amp_pd_peptide
env = amp_pd_peptide.make_env()
iter_test = env.iter_test()

for (test, test_peptides, test_proteins, sample_submission) in iter_test:
    
    targets = sample_submission.prediction_id.str.split('_').apply(lambda x: (int(x[1]) + int(x[5]) , '_'.join(x[2:4])))
    sample_submission['rating'] = targets.map(estimates)
    
    env.predict(sample_submission)

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.


In [21]:
submission = pd.read_csv('/kaggle/working/submission.csv')
submission

,prediction_id,rating
0,3342_0_updrs_1_plus_0_months,5.0
1,3342_0_updrs_1_plus_6_months,6.0
2,3342_0_updrs_1_plus_12_months,6.0
3,3342_0_updrs_1_plus_24_months,6.0
4,3342_0_updrs_2_plus_0_months,4.0
...,...,...
59,50423_6_updrs_3_plus_24_months,21.0
60,50423_6_updrs_4_plus_0_months,0.0
61,50423_6_updrs_4_plus_6_months,0.0
62,50423_6_updrs_4_plus_12_months,0.0
